In [1]:
!pip install numpy
!pip install tensorflow
!pip install keras
!pip install keras-pos-embd
!pip install keras-multi-head
!pip install keras-layer-normalization
!pip install keras-position-wise-feed-forward
!pip install keras-embed-sim

In [2]:
from __future__ import unicode_literals

In [13]:
import unittest
import numpy as np
from Keras_performer import performer as tfr
import nltk
from os import listdir
from os.path import isfile, isdir, join

ModuleNotFoundError: No module named 'backend'

In [4]:
nltk.download('punkt')

NameError: name 'nltk' is not defined

In [5]:
import csv

In [6]:
def readCSV(file_name):
  errlist=[]
  LBlist=[]
  with open(file_name, newline='') as csvfile:

  # 讀取 CSV 檔內容，將每一列轉成一個 dictionary

    rows = csv.DictReader(csvfile)
    for row in rows: 
      RL=list(row.values())

      RL = list(map(int, RL))
      errs=RL[1:37]
      LB=RL[37:]
      errlist.append(errs)
      LBlist.append(LB)
  return errlist,LBlist

In [7]:
def parseSentence(x):	
	tokens=[]
	state="START"
	chrs=""
	for i in range(len(x)):
		#print(ord(x[i]))
		if (ord(x[i])>255):
			inp="U"
		else:
			inp="E"
	
		if state=="START":
			if inp=="E":
				state="ASCII"
				chrs=x[i]
			else:
				state="UNICODE"
				tokens.append(x[i])
			
		elif state=="ASCII":
			if inp=="E":
				chrs += x[i]
			else:#U
				state="UNICODE"
				tokens += nltk.word_tokenize(chrs)
				chrs=""
				tokens.append(x[i])
	
		elif state=="UNICODE":
			if inp=="E":
				state="ASCII"
				chrs=x[i]
			else:
				state="UNICODE"
				tokens.append(x[i])
	if len(chrs)>0:
		tokens += nltk.word_tokenize(chrs) 
	return tokens


In [8]:
def readcode(fname):
    with open(fname) as f:
        data = f.read()
        return data


In [9]:
class TestTranslate(unittest.TestCase):
 
    @staticmethod
    def _build_token_dict(token_list):
        token_dict = {
            '<PAD>': 0,
            '<START>': 1,
            '<END>': 2,
        }
        for tokens in token_list:
            for token in tokens:
                if token not in token_dict:
                    token_dict[token] = len(token_dict)
        return token_dict
 
    def test_translate(self):
        print("i am here: " )
        source_tokens = []
        target_errors=[]
        target_LB=[]
        #source_file=[]
        Input_Path = "/content/drive/MyDrive/AI_JAVA_ANNOTATION_PROJ_2021/訓練資料產生目錄/訓練資料/INPUT"
        Location_Output_Path = "/content/drive/MyDrive/AI_JAVA_ANNOTATION_PROJ_2021/訓練資料產生目錄/訓練資料/錯誤段落_Output"
 
        InputFiles = sorted(listdir(Input_Path))
        OutputFiles = sorted(listdir(Location_Output_Path))
        #max_files =10
        for f in InputFiles:
          fullpath = join(Input_Path,f)
          #source_file=fullpath
          if isfile(fullpath):
            source_tokens.append(parseSentence(readcode(fullpath)))
            #if len(source_tokens)>max_files: break
        for f in OutputFiles:
          fullpath = join(Location_Output_Path,f)
          #source_file=fullpath
          if isfile(fullpath):
            err,lb = readCSV(fullpath)
            target_errors.append(err)
            target_LB.append(lb)
            #if len(source_tokens)>max_files: break
        dd=np.asarray(target_errors)
        print("AAAA: ", dd.shape)
        print("aaaa: " , type(target_errors[0][0]))
        target_errors=target_errors[0]  
        target_LB=target_LB[0]     
        print("XXXX: " , len(source_tokens))
        print("YYYY: " , type(target_errors[0][0]))
        print("ZZZZ: " , len(target_LB))

 
        source_tokens2 = []
        target_errors2= []
        target_LB2=[]

        THRESHOLD_FILE_LEN = 64000000
 
        for i in range(len(source_tokens)): 
          #print (i)
          src = source_tokens[i]
          err = target_errors[i]   
          lb = target_LB[i]                   
          if (len(src)<=THRESHOLD_FILE_LEN):
            source_tokens2.append(src)
            target_errors2.append(err)
            target_LB2.append(lb) 
        source_tokens = source_tokens2
        target_errors= target_errors2        
        target_LB= target_LB2        

        print("XXXX2: " , len(source_tokens))
                    
 
        # Generate dictionaries
        source_token_dict = self._build_token_dict(source_tokens)
        
 
        # Add special tokens
        encode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
        
        #output_tokens = [tokens + ['<END>', '<PAD>'] for tokens in target_tokens]
 
        # Padding
        sl = max(map(len, encode_tokens))
        
        source_max_len = sl
        
 
        encode_tokens = [tokens + ['<PAD>'] * (source_max_len - len(tokens)) for tokens in encode_tokens]
        
        
 
        encode_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in encode_tokens]
       
        
        #print("encode_input:", encode_input)
        
        #print("len(source_token_dict):", len(source_token_dict))
        
        
        #decode_output_one_hot_encoded = to_categorical(decode_output)
        #print("one-hot-decode_output:", decode_output_one_hot_encoded)
        #decode_output =  list(decode_output_one_hot_encoded)
        token_num =len(source_token_dict)
        print(type(token_num))
         # Build & fit model
        model = tfr.get_model(
            input_len= (source_max_len),
            errNum=36,
            lbNum=192,
            token_num=len(source_token_dict),
            embed_dim=32,
            encoder_num=2,
            head_num=4,
            hidden_dim=128,
            dropout_rate=0.05
        )
        losses = {    
            "error_feed_forward_output1": "categorical_crossentropy",
            "error_feed_forward_output2": "mean_squared_error",            
        }
        lossWeights = {"error_feed_forward_output1": 1.0,"error_feed_forward_output2": 1.0}
        model.compile('adam', loss=losses, loss_weights=lossWeights)
        
        print("x.shape", np.asarray(encode_input).shape)  #x.shape (2,  9)
        
 
        x=[np.array(encode_input * 1)]
        y=[np.array(target_errors * 1),np.array(target_LB * 1)]
       
        print("x.shape", np.asarray(x).shape)  #x.shape (2, 2048, 9)
        
        model.fit(
            x=x,
            y=y,
            epochs=2,
            batch_size=32,
        )
        model.save("test_model.h5")
        saveDictionary(source_token_dict, 'source_token_dict.pickle')
        
        
        
        # Predict 
        decoded = tfr.decode(
            model,
            encode_input,
            start_token=target_token_dict['<START>'],
            end_token=target_token_dict['<END>'],
            pad_token=target_token_dict['<PAD>'],
            max_len=source_max_len
        )
        
 
        for i in range(len(encode_input)):
          print("=====標準答案=====", ''.join(target_tokens[i]))
          predicted = ''.join(map(lambda x: target_token_dict_inv[x], decoded[i][1:-1]))
          print("=====預測答案=====", predicted)

In [10]:
x=TestTranslate()
x.test_translate()

i am here: 


NameError: name 'listdir' is not defined